## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
dbutils.widgets.text(name="storage_account", defaultValue= " ", label= "Storage Account String")
dbutils.widgets.text(name="container_name", defaultValue= " ", label= "container Name")
dbutils.widgets.text(name="mount_path",defaultValue=" ",label="Mount Path")
dbutils.widgets.text(name="storage_account_key", defaultValue= "Ends with ==", label= "Storage Account key")
dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
import pyspark.sql.functions as f

In [0]:
# File location and type
file_location = "/FileStore/tables/EmployeeData/CpuLogData*.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.select("DateTime","keyboard","mouse","user_name")\
        .withColumn('Date', f.split(df['DateTime'], ' ').getItem(0))\
        .withColumn('Time', f.split(df['DateTime'], ' ').getItem(1))\
        .withColumn('DateTime', f.to_timestamp(df['DateTime']))
print(df.count())



4122

In [0]:
# Create a view or table
temp_table_name = "CpuLogData"
df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql
/* Query the created temp table in a SQL cell */
select count(*) from `CpuLogData`

count(1)
4122


In [0]:
storage_account_name =dbutils.widgets.get("storage_account")
# Azure Storage Account Key
storage_account_key =dbutils.widgets.get("storage_account_key")

# Azure Storage Account Source Container
container = dbutils.widgets.get("container_name")
mount_path=dbutils.widgets.get("mount_path")
# Set the configuration details to read/write
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [0]:
dbutils.fs.mount(
  source = url,
  mount_point = "/mnt/"+mount_path,
  extra_configs = {config:sas}
)

# python
#df = spark.read.text("/mnt/"+mountName+"/raw_data.json")
#df = spark.read.option("multiline", "true").json("/mnt/"+mountName)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2916639903946428> in <module> 
 10 source = url , 
 11 mount_point = "/mnt/" + mountName , 
 ---> 12 extra_configs = { config : sas } 
 13 )
 14 

 /local_disk0/tmp/1614141210601-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 322 exc . __context__ = None 
 323 exc . __cause__ = None 
 --> 324 raise exc
 325 return f_with_exception_handling
 326 

 ExecutionError : An error occurred while calling o340.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/mount1; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/mount1
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:470)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/mount1
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:404)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:670)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:504)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:659)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:412)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:89)
	at com.databricks.backend.daemon.data.server.handler.CEMountHandler.receive(MountHandler.scala:141)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:288)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:248)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.loggin

In [0]:
''' Query to Find users with lowest number of average hours and highest number of average hours '''

avg_active_hrs  = sqlContext.sql("select user_name,from_unixtime(ROUND((count(*)*5)*60/6,2),'HH:mm') as avg_active_hrs from `CpuLogData` where keyboard!=0 or mouse!=0 group by user_name order by avg_active_hrs desc")
display(avg_active_hrs)
#avg_active_hrs.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true") .option("inferSchema", "false").option("delimiter", ",").save(url+"/logging/active_hrs.csv")

user_name,avg_active_hrs
deepshukla292@gmail.com,06:35
iamnzm@outlook.com,06:22
sharlawar77@gmail.com,06:20
salinabodale73@gmail.com,06:06
rahilstar11@gmail.com,05:32
markfernandes66@gmail.com,05:24
bhagyashrichalke21@gmail.com,05:00
damodharn21@gmail.com,02:39


In [0]:
''' Query to Find users with highest number of idle hours '''

avg_inactive_hrs = sqlContext.sql("select user_name,from_unixtime(ROUND((count(*)*5)*60/6,2),'HH:mm') as inactive_hrs from `CpuLogData` where keyboard=0 and mouse=0 group by user_name order by inactive_hrs desc")
display(avg_inactive_hrs)
#avg_inactive_hrs.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true") .option("inferSchema", "false").option("delimiter", ",").save(url+"/logging/inactive_hrs.csv")

user_name,inactive_hrs
iamnzm@outlook.com,02:09
rahilstar11@gmail.com,02:06
salinabodale73@gmail.com,01:47
sharlawar77@gmail.com,01:42
bhagyashrichalke21@gmail.com,01:40
markfernandes66@gmail.com,01:39
deepshukla292@gmail.com,01:15
damodharn21@gmail.com,00:51


In [0]:
daily_session_start_time_df = sqlContext.sql("select Date,to_timestamp(min(DateTime)) as DateTime from CpuLogData group by Date order by Date")
temp_table_name = "view_session_start_time"
daily_session_start_time_df.createOrReplaceTempView(temp_table_name)
display(daily_session_start_time_df)


Date,DateTime
2019-09-16,2019-09-16T12:55:01.000+0000
2019-09-17,2019-09-17T08:25:01.000+0000
2019-09-18,2019-09-18T08:30:01.000+0000
2019-09-19,2019-09-19T08:40:02.000+0000
2019-09-20,2019-09-20T09:05:01.000+0000
2019-09-21,2019-09-21T09:10:01.000+0000


In [0]:
daily_login_time_df = sqlContext.sql("select user_name,Date,min(DateTime) as Datetime from `CpuLogData` where (keyboard != 0 or mouse != 0) group by user_name,Date order by Date")
temp_table_name = "view_daily_login_time"
daily_login_time_df.createOrReplaceTempView(temp_table_name)
display(daily_login_time_df)
#daily_login_time_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true") .option("inferSchema", "false").option("delimiter", ",").save(url+"/logging/active.csv")

user_name,Date,Datetime
deepshukla292@gmail.com,2019-09-16,2019-09-16T13:00:01.000+0000
salinabodale73@gmail.com,2019-09-16,2019-09-16T12:55:02.000+0000
rahilstar11@gmail.com,2019-09-16,2019-09-16T13:00:03.000+0000
iamnzm@outlook.com,2019-09-16,2019-09-16T13:00:01.000+0000
bhagyashrichalke21@gmail.com,2019-09-16,2019-09-16T12:55:01.000+0000
sharlawar77@gmail.com,2019-09-16,2019-09-16T13:00:04.000+0000
markfernandes66@gmail.com,2019-09-17,2019-09-17T10:50:01.000+0000
deepshukla292@gmail.com,2019-09-17,2019-09-17T09:30:01.000+0000
sharlawar77@gmail.com,2019-09-17,2019-09-17T10:45:02.000+0000
salinabodale73@gmail.com,2019-09-17,2019-09-17T10:15:01.000+0000


In [0]:
''' Query to find the highest number to times users late coming '''

late_users_df = sqlContext.sql("select lut.user_name,Concat(cast(Round((Sum(lut.TimeDifference)/60)/6,2) AS INT), 'Hour :', Floor(Round((((Sum(lut.TimeDifference)/60)/6)%1)*60)), 'Min') as average_late_time_in_hrs,count(*) as num_of_late_comings from(Select lt.user_name,(unix_timestamp(lt.DateTime)-unix_timestamp(st.DateTime))/(60) as TimeDifference from view_daily_login_time as lt,view_session_start_time as st where lt.Date == st.Date) as lut where lut.TimeDifference != 0 group by lut.user_name order by average_late_time_in_hrs desc")
display(late_users_df)
#spark.conf.set("fs.azure.account.key."+containerName+".blob.core.windows.net","P0Pm8ZZf1LcQu1PXJfhVv1OUGDmkrg1Btf1OwJ8wpLovqOq1RzQhmFmLKObQshkgR5IRRE2oUfSJpKEqUCcgkA==")
#late_users_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true") .option("inferSchema", "false").option("delimiter", ",").save(url+"/logging/latecomings.csv")

user_name,average_late_time_in_hrs,num_of_late_comings
markfernandes66@gmail.com,1Hour :9Min,5
sharlawar77@gmail.com,1Hour :8Min,5
salinabodale73@gmail.com,1Hour :27Min,6
bhagyashrichalke21@gmail.com,1Hour :27Min,5
rahilstar11@gmail.com,1Hour :23Min,6
damodharn21@gmail.com,0Hour :59Min,3
iamnzm@outlook.com,0Hour :34Min,4
deepshukla292@gmail.com,0Hour :22Min,4
